In [4]:
#抓取新聞連結
def getlink(script):
    arry = [] 
    global count
    global pkcount 
    apl = 'apl'
    soup = BeautifulSoup(res.text)
    for a_tag in soup.select('#result h2 a'):
        alink = a_tag.get('href').strip()
        arry.append(alink)
        for elelink in arry:
            num = str(pkcount)
            if pkcount < 10:
                num = str(0) * 4 + num
            elif pkcount < 100:     
                num = str(0) * 3 + num
            elif pkcount < 1000:     
                num = str(0) * 2 + num
            elif pkcount < 10000:     
                num = str(0) * 1 + num
            pk = apl + num
        links = {'link':alink,'area':area,'page':page,'count':count,'pk':pk}
        linkary.append(links)
        count += 1
        pkcount += 1

In [5]:
import requests
from datetime import *
from bs4 import BeautifulSoup
import time

#12個行政區['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':'','pk':''}
linkary =[]
    
#系統時間------------------------------
today = date.today()
year = today.year
month = today.month
day = today.day
if month < 10:
    month = "0" + str(month)
if day < 10:
    day = "0" + str(day)
year = str(year)
month = str(month)
day = str(day)
#--------------------------------------

headers={
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36' 
}
pkcount = 1    # pkcount = 1
for area in ary:
    count = 1
    for page in range(1,9):
        payload = {
        'searchMode':'',
        'searchType':'text',
        'ExtFilter':'',
        'sorttype':'1',
        'keyword':area,
        'rangedate':'[20030502 TO'+year+month+day+'999:99'']',
        'totalpage':'',
        'page':page
        }

        res = requests.post('http://search.appledaily.com.tw/appledaily/search',headers=headers,data=payload)
        script = res.text
        getlink(script)

In [6]:
import pandas
data1 = pandas.DataFrame(linkary)
data1

,area,count,link,page,pk
0,中正區,1,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00001
1,中正區,2,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00002
2,中正區,3,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00003
3,中正區,4,http://www.appledaily.com.tw/appledaily/articl...,1,apl00004
4,中正區,5,http://www.appledaily.com.tw/appledaily/articl...,1,apl00005
5,中正區,6,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00006
6,中正區,7,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00007
7,中正區,8,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00008
8,中正區,9,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00009
9,中正區,10,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00010


In [8]:
#輸出成CSV
data1.to_csv('apl_link.csv',encoding='utf-8')

In [10]:
#讀取CSV檔
import csv
linkary = []
with open('apl_link.csv',) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

In [11]:
#抓取新聞內文
import requests
import time
import re  # --
from bs4 import BeautifulSoup

headers = { 
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36'
}

for ele in linkary:
    url = ele['link']
    text = ''
    while True:
        try:
            res = requests.get(url,headers=headers)
            res.text
            soup = BeautifulSoup(res.text)
            break
        except:
            time.sleep(5)
            
    try:
#         if soup.select('.urcc a') is not None:
#             ele['title'] = soup.select('hgroup #h1')[0].text.strip()
#             ele['date'] = soup.select('.gggs time')[0]['datetime']
#             reele = soup.select('.urcc a')[0].text
#             rerx = re.search('(\d+)',reele)
#             ele['like'] = rerx.group(1)
#         else:
#             ele['title'] = soup.select('hgroup #h1')[0].text.strip()
#             ele['date'] = soup.select('.gggs time')[0]['datetime']
#             ele['like'] = '0'
#             for p_tag in soup.select('#summary'):
#                 text += p_tag.text.strip()
#             ele['content'] = text
        
        ele['title'] = soup.select('hgroup #h1')[0].text.strip()
        ele['date'] = soup.select('.gggs time')[0]['datetime']
        for p_tag in soup.select('#summary'):
            text += p_tag.text.strip()
        ele['content'] = text
        
    except:
        print "This link has some problems!"
        print ele['link'], ele['area'], ele['count']

This link has some problems!
http://www.appledaily.com.tw/appledaily/article/property/20151011/36830793/applesearch/名人投資GiGi林如琦房產與副業多元投資 中山區 42
This link has some problems!
http://www.appledaily.com.tw/appledaily/article/property/20151011/36830793/applesearch/名人投資GiGi林如琦房產與副業多元投資 大安區 56


In [12]:
#新聞內文進入DataFrame
import pandas
data2 = pandas.DataFrame(linkary)

# 移除多餘的 column
# del data2['column name']
apl_ok = data2.drop(data2.columns[[0]], axis=1)
apl_ok

,area,content,count,date,link,page,pk,title
0,中正區,第13屆台灣同志大遊行今天登場，力挺婚姻平權的第三勢力也紛紛支持，時代力量、綠社盟都分別帶隊...,1,2015/10/31/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00001,挺同志遊行 綠社盟：當選立委後推婚姻平權
1,中正區,雙層店面好運用，價格水漲船高，房仲統計近1年雙北店面交易實價，台北市1樓店面年減0.6%、新...,2,2015/10/27/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00002,雙層店面夯 北市價格年漲近2成
2,中正區,台北市衛生局今公布新增1例本土登革熱個案。新增本土個案一名為中正區26歲男子，本月16日至1...,3,2015/10/27/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00003,北市今增1例本土登革熱 患者曾到過台南
3,中正區,,4,2015/10/27/,http://www.appledaily.com.tw/appledaily/articl...,1,apl00004,博愛特區搶銀行 匪持槍打傷保全
4,中正區,,5,2015/10/27/,http://www.appledaily.com.tw/appledaily/articl...,1,apl00005,國有地上權標售冷 僅2筆標出
5,中正區,（更新：新增嫌搶匪所乘計程車、逃逸路線）博愛特區驚傳銀行搶案！今天下午2時許，一名男子跑到台...,6,2015/10/26/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00006,博愛特區銀行搶案 歹徒所搭計程車現蹤
6,中正區,國產署今北中南同步進行地上權土地招標，13筆僅標出2筆。 這次招標面積共38162.72平方...,7,2015/10/26/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00007,溢價6% 北市逾200坪地上權地標出
7,中正區,亮槍恐嚇被判刑2月！基隆市33歲陳姓男子，開車行經中正區新豐街，因不滿後方車輛跟車太近，似乎...,8,2015/10/25/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00008,不給超車還亮槍 男子恐嚇遭判刑2月
8,中正區,本內容由公庫提供文／法律扶助基金會時間：105/10/23（五）早上09：00地點：台北地方...,9,2015/10/22/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00009,【公庫】為起訴而起訴？反課綱學生被起訴聲援
9,中正區,本內容由公民監督國會聯盟提供第九屆立法委員選舉，除了傳統國、民兩大黨，更有許多小黨紛紛投入參...,10,2015/10/22/,http://www.appledaily.com.tw/realtimenews/arti...,1,apl00010,【公督盟】監督職人教你選出好立委 讓你四年不後悔!


In [13]:
#輸出完成品
apl_ok.to_csv('apl_ok.csv',encoding='utf-8')